## CSCI 470 Activities and Case Studies

1. For all activities, you are allowed to collaborate with a partner. 
1. For case studies, you should work individually and are **not** allowed to collaborate.

By filling out this notebook and submitting it, you acknowledge that you are aware of the above policies and are agreeing to comply with them.

Some considerations with regard to how these notebooks will be graded:

1. Cells in which "# YOUR CODE HERE" is found are the cells where your graded code should be written.
2. In order to test out or debug your code you may also create notebook cells or edit existing notebook cells other than "# YOUR CODE HERE". We actually highly recommend you do so to gain a better understanding of what is happening. However, during grading, **these changes are ignored**. 
2. You must ensure that all your code for the particular task is available in the cells that say "# YOUR CODE HERE"
3. Every cell that says "# YOUR CODE HERE" is followed by a "raise NotImplementedError". You need to remove that line. During grading, if an error occurs then you will not receive points for your work in that section.
4. If your code passes the "assert" statements, then no output will result. If your code fails the "assert" statements, you will get an "AssertionError". Getting an assertion error means you will not receive points for that particular task.
5. If you edit the "assert" statements to make your code pass, they will still fail when they are graded since the "assert" statements will revert to the original. Make sure you don't edit the assert statements.
6. We may sometimes have "hidden" tests for grading. This means that passing the visible "assert" statements is not sufficient. The "assert" statements are there as a guide but you need to make sure you understand what you're required to do and ensure that you are doing it correctly. Passing the visible tests is necessary but not sufficient to get the grade for that cell.
7. When you are asked to define a function, make sure you **don't** use any variables outside of the parameters passed to the function. You can think of the parameters being passed to the function as a hint. Make sure you're using all of those variables.
8. Finally, **make sure you run "Kernel > Restart and Run All"** and pass all the asserts before submitting. If you don't restart the kernel, there may be some code that you ran and deleted that is still being used and that was why your asserts were passing.

# Model Interpretability

In this exercise you'll use the [alibi](https://docs.seldon.io/projects/alibi/en/stable/) library to explain why some models make the predictions they do.

In [ ]:
# Uncomment to install the library

# ! pip install alibi

In [ ]:
import sklearn
from sklearn.datasets import load_iris
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score

## Load and examine the Iris dataset

In [ ]:
data = load_iris()

In [ ]:
print(data["DESCR"])
features = data["data"]
targets = data["target"]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(features, targets, random_state=0)

In [ ]:
print(len(X_test))

## Build two different classifiers and train them on the Iris data

In [ ]:
# Create 2 classifiers: a Random Forest model named "rf_clf", and a Linear SVM model named "svm_clf"
#
# Train them both on the training data.
# Use them to predict the test data - saving predictions as "y_rf_pred" and "y_svm_pred".
# You may want to use GridSearchCV to determine hyperparameters.

# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
assert len(y_rf_pred) == 38
assert isinstance(rf_clf, RandomForestClassifier) or isinstance(rf_clf, GridSearchCV)
assert len(y_svm_pred) == 38
assert isinstance(svm_clf, LinearSVC) or isinstance(svm_clf, GridSearchCV)

In [ ]:
print(f"Random forest test set accuracy: {accuracy_score(y_test, y_rf_pred):0.3f}.")
print(f"SVM test set accuracy:           {accuracy_score(y_test, y_svm_pred):0.3f}.")

## Get prediction explanations

In [ ]:
# Since we used a Linear SVM, we can easily determine the coefficients for the features:

print("Each class gets a coefficient for each feature that helps us determine that feature's importance.")

print('\nSVM feature coefficients:')
if isinstance(svm_clf, LinearSVC):
    print(svm_clf.coef_)
elif isinstance(svm_clf, GridSearchCV):
    print(svm_clf.best_estimator_.coef_)

Now let's look at how we can use explainers, namely the [AnchorTabular](https://docs.seldon.io/projects/alibi/en/stable/methods/Anchors.html#id3) explainer to understand why the models make the predictions they do.

In [ ]:
from alibi.explainers import AnchorTabular

Alibi explainers follow a general structure of:

1. Initialize the explainer, providing a prediction function, and explainer specific parameters. `exp = Explainer(predict_func, param_1, param_2, ...)`
1. Fit the explainer to the training data (this step is explainer dependent) `exp.fit(train_data)`
1. Explain a given sample `exp.explain(sample)`

First, we reframe the prediction pipeline into a prediction function that we can use with the explainer:

In [ ]:
rf_clf_func = lambda x: rf_clf.predict(x)
svm_clf_func = lambda x: svm_clf.predict(x)

Now we can instantiate the explainer using the prediction function and any parameters the explainer requires:

In [ ]:
rf_explainer = AnchorTabular(rf_clf_func, data["feature_names"])
rf_explainer.fit(X_train)

In [ ]:
svm_explainer = AnchorTabular(svm_clf_func, data["feature_names"])
svm_explainer.fit(X_train)

Once the explainer is set up, we can now use it to `.explain` samples! Pick a sample below to explain the two models' predictions.

In [ ]:
# Change this value to choose a test sample
index_to_explain = 7


rf_explanation = rf_explainer.explain(X_test[index_to_explain])
svm_explanation = svm_explainer.explain(X_test[index_to_explain])

In [ ]:
rf_explanation.anchor, rf_explanation.precision

In [ ]:
svm_explanation.anchor, svm_explanation.precision

Here we can see what the model's explanation for the classification of that sample is. You can see that even with our relatively interpretable model of Linear SVMs, these explainers can provide a more direct and intuitive explanation for why a sample was labeled the way it was.

Now that you've seen the general approach for these explainers, let's work on something a bit more complex. Now you'll have to create the models, the prediction function, and the explainers.

## Load the MNIST dataset

Explaining data from measured observations is simple enough. Now let's try explaining how images get labeled.

In [ ]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
import matplotlib
import matplotlib.pyplot as plt
import numpy as np

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train/255.
x_test = x_test/255.

### Add noise to the MNIST images

Because the nearly binary (black and white) nature of the MNIST images, the formation of superpixels is not representative of the type of behavior you would get from more complex, color images. To get closer to that type of behavior, we'll add noise to the images.

In [ ]:
# Add noise to the MNIST images

x_train = x_train/2 + np.random.uniform(0, high=0.5, size=x_train.shape)
x_train = np.clip(x_train, 0, 1)

x_test = x_test/2 + np.random.uniform(0, high=0.5, size=x_test.shape)
x_test = np.clip(x_test, 0, 1)

In [ ]:
sample_index = 12
sample = x_train[sample_index]

In [ ]:
_ = plt.imshow(sample, cmap="gray")

## Build a CNN classifier and train it on MNIST

In [ ]:
# Create a neural network model that should do well on the MNIST dataset and save it to mnist_nn
# Make the neural network sufficiently complex (at least 4 neural layers--max pooling and flatten
# layers do not count). A CNN is recommended over a fully-connected feedforward network.

# Save the neural network as "mnist_nn"
#
# *** Make sure you get at least 80% accuracy ***

# YOUR CODE HERE
raise NotImplementedError()

mnist_nn.summary()

mnist_nn.compile(optimizer="adam", loss='sparse_categorical_crossentropy', metrics=['accuracy'])
mnist_nn.fit(x_train.reshape(-1, 28, 28 ,1), y_train,
             validation_data=(x_test.reshape(-1, 28, 28 ,1), y_test),
             epochs=1)

In [ ]:
assert len(mnist_nn.layers) >= 4
assert mnist_nn.evaluate(x_test.reshape(-1, 28, 28 ,1), y_test)[1] > 0.8

## Get explanations for the MNIST model's predictions

To work with images, we'll use the [AnchorImage](https://docs.seldon.io/projects/alibi/en/stable/methods/Anchors.html#id5) explainer. This explainer requires that we break up the image into "superpixels". We'll use the default 'slic' method to do so.

In [ ]:
from alibi.explainers import AnchorImage

In [ ]:
# Create an explainer object using AnchorImage that explains the mnist_nn model you created.
# Use the 'slic' method for as the segmentation function.
# Save the explainer as "mnist_explainer".

# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
assert isinstance(mnist_explainer, AnchorImage)

In [ ]:
x_test.shape

In [ ]:
## Get an image, a prediction, and an explanation


# Change this number to try out different samples
image_index_to_explain = 6


# Get the image
image_to_explain = x_test[image_index_to_explain].reshape(28, 28, 1)

# Get the prediction
pred = mnist_nn.predict(image_to_explain.reshape(1, 28, 28, 1)).argmax()

# Get the explanation
mnist_image_explanation = mnist_explainer.explain(image_to_explain, threshold=0.95, p_sample=0.5)

# Show results
plt.figure(figsize=(15, 5))
plt.subplot(1, 3, 1)
plt.imshow(image_to_explain[:,:,0], cmap="gray")
plt.title('Image')

plt.subplot(1, 3, 2)
plt.imshow(mnist_image_explanation.segments)
plt.title('Segments')

plt.subplot(1, 3, 3)
plt.imshow(mnist_image_explanation.anchor[:,:,0], cmap="gray")
_ = plt.title(f'Anchor: Pixels that explain prediction of {pred}')

### Impact of segmentation

One thing you may have noticed is that the explanations are heavily dependent on the superpixels we identify. 

You may want to alter some of the parameters of the 'slic' segmentation function to see how it impacts the segmentation, and thus the explaining anchor. Add the keyword argument below to your `AnchorImage` instantiation above, and adjust the values.  
`segmentation_kwargs={'n_segments': 15, 'compactness': 20, 'sigma': .5}`


## Feedback

In [ ]:
def feedback():
    """Provide feedback on the contents of this exercise
    
    Returns:
        string
    """
    # YOUR CODE HERE
    raise NotImplementedError()